# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import os
import joblib
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.automl import AutoMLConfig

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AQ92RUM85 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
quick-starts-ws-137154
aml-quickstarts-137154
southcentralus
5a4ab2ba-6c51-4805-8155-58759ad589d8


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
#create compute cluster
amlcompute_cluster_name="aml"

#verify that cluster does not already exist
try:
    aml_compute=ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config=AmlCompute.provisioning_configuration(vm_size="Standard_D3_V2",max_nodes=4)
    aml_compute=ComputeTarget.create(ws,amlcompute_cluster_name,compute_config)

aml_compute.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [3]:
# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
auto_ds_path="https://raw.githubusercontent.com/shat700/nd00333-capstone/master/starter_file/heart_failure_clinical_records_dataset.csv"
ds=TabularDatasetFactory.from_delimited_files(path=auto_ds_path)

#preview data
df = ds.to_pandas_dataframe()
df.describe()



,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [4]:
import logging
import time

# TODO: Put your automl settings here
automl_settings = {
    "n_cross_validations":5,
    "primary_metric":'accuracy',
    "enable_early_stopping":True,
    "experiment_timeout_minutes":15,
    "max_concurrent_iterations":10,
   }

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task='classification',
    compute_target='aml',
    training_data=ds,
    label_column_name='DEATH_EVENT',
    **automl_settings
    )

In [5]:
from azureml.core.experiment import Experiment
# choose a name for experiment
experiment_name = 'automl_run'
experiment=Experiment(ws, experiment_name)

# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on aml with default configuration
Running on remote compute: aml
Parent Run ID: AutoML_d47dac20-ceff-4bc6-8635-4a0673dad3bc

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/A

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_d47dac20-ceff-4bc6-8635-4a0673dad3bc',
 'target': 'aml',
 'status': 'Completed',
 'startTimeUtc': '2021-02-04T00:21:58.271044Z',
 'endTimeUtc': '2021-02-04T00:34:11.227673Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'aml',
  'AMLSettingsJsonString': '{"path":null,"name":"automl_run","subscription_id":"5a4ab2ba-6c51-4805-8155-58759ad589d8","resource_group":"aml-quickstarts-137154","workspace_name":"quick-starts-ws-137154","region":"southcentralus","compute_target":"aml","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":null,"featurization":"auto","_igno

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [7]:
# Retrieve and save your best automl model.
best_remote_run,best_automl_model=remote_run.get_output()
best_run_metrics=remote_run.get_metrics()
for primary_metric in best_run_metrics:
    metric=best_run_metrics[primary_metric]
    print(primary_metric,metric)


Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


experiment_status ['DatasetEvaluation', 'FeaturesGeneration', 'DatasetFeaturization', 'DatasetFeaturizationCompleted', 'DatasetCrossValidationSplit', 'ModelSelection']
experiment_status_description ['Gathering dataset statistics.', 'Generating features for the dataset.', 'Beginning to fit featurizers and featurize the dataset.', 'Completed fit featurizers and featurizing the dataset.', 'Generating individually featurized CV splits.', 'Beginning model selection.']
AUC_micro 0.9203936767850873
recall_score_micro 0.882994350282486
recall_score_weighted 0.882994350282486
f1_score_micro 0.882994350282486
precision_score_weighted 0.8993850125842966
weighted_accuracy 0.9028878829357401
average_precision_score_macro 0.9038740695599777
average_precision_score_micro 0.9225894262484499
log_loss 0.36612847580299307
f1_score_weighted 0.8787729930147679
AUC_macro 0.9198838362864524
accuracy 0.882994350282486
norm_macro_recall 0.7063095238095238
recall_score_macro 0.8531547619047618
f1_score_macro 0.

In [8]:
print(best_remote_run)
best_remote_run

Run(Experiment: automl_run,
Id: AutoML_d47dac20-ceff-4bc6-8635-4a0673dad3bc_48,
Type: azureml.scriptrun,
Status: Completed)


Experiment,Id,Type,Status,Details Page,Docs Page
automl_run,AutoML_d47dac20-ceff-4bc6-8635-4a0673dad3bc_48,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [9]:
#TODO: Save the best model
import pickle
best_remote_run,best_automl_model=remote_run.get_output()
#joblib.dump(value=best_automl_model,filename='automl_model.joblib')
pkl_filename = "automl_model.pkl"
with open(pkl_filename,'wb') as f:
    pickle.dump(best_automl_model,f)

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [10]:
from azureml.core.model import Model
# Register model
model = Model.register(workspace = ws,
                        model_name = "Heart_failure_prediction",
                        model_path = "./automl_model.pkl",
                        description = "Best AutoML Model",)



Registering model Heart_failure_prediction


In [13]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies


#defining an inference configuration
service_name = 'heart-failure-automl'

#setting up a curated environment
from azureml.automl.core.shared import constants
best_remote_run.download_file(constants.CONDA_ENV_FILE_PATH, 'AzureML-AutoML.yml')
env = Environment.get(ws, name="AzureML-AutoML")

#combining scoring script and environment in inference configuration
inference_config = InferenceConfig(entry_script='scoring.py', environment=env)

#deploy the model as a webservice
from azureml.core.webservice import LocalWebservice, Webservice

#set deployment configuration
local_deployment_config = LocalWebservice.deploy_configuration(port=8000)
service = Model.deploy(ws, "heart-failure-automl", [model], inference_config, local_deployment_config)

print(service.state)

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, enable_app_insights=True, auth_enabled=True) #service.update

service = Model.deploy(workspace=ws,
                        name='heart-failure-automl',
                        models=[model],
                        inference_config=inference_config,
                        deployment_config=aci_config,
                        overwrite=True)
service.wait_for_deployment(show_output=True)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry viennaglobal.azurecr.io
Logging into Docker registry viennaglobal.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM viennaglobal.azurecr.io/azureml/azureml_1eed3e6a679bbc599bfa3e074a25301a
 ---> 59311b131e1a
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 987147278179
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjVhNGFiMmJhLTZjNTEtNDgwNS04MTU1LTU4NzU5YWQ1ODlkOCIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTEzNzE1NCIsImFjY291bnROYW1lIjoicXVpY2stc3RhcnRzLXdzLTEzNzE1NCIsIndvcmtzcGFjZUlkIjoiNGJiZDIxYTktNTI3NS00ZDZhLTgwMmMtNGE4MDg4MWZhMjJlIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 1e2681c28e88
 ---> 9c9cb2489545
Step 4/5 : RUN mv '/var/azureml-app/tmp9j0unxrc.py' /var/azureml-app/main.py
 ---> Running in 0532461388ef
 ---> 3792951b9aad
Step 5/5 : 

In [45]:
#retrieve deployed model uri
print(service.scoring_uri)
print(service.swagger_uri)

http://d055feb8-5c29-4fe8-acee-961ead8fd279.southcentralus.azurecontainer.io/score
http://d055feb8-5c29-4fe8-acee-961ead8fd279.southcentralus.azurecontainer.io/swagger.json


In [46]:
#retrive primary key
primary, secondary= service.get_keys()
print(primary)

MUJAeBBGnn5ur078rydX7OGdjpMgOFOH


TODO: In the cell below, send a request to the web service you deployed to test it.

In [47]:
#Testing of the deployed webservice
import requests
import json
import pandas as pd
import numpy as np

# URL for the web service
scoring_uri = service.scoring_uri
# If the service is authenticated, set the key or token
key = primary

# Two sets of data to score, so we get two results back
data = {"data":
        [
            {
               "age":55,
               "anaemia":0,
               "creatinine_phosphokinase":7861,
               "diabetes":0,
               "ejection_fraction":38,
               "high_blood_pressure":0,
               "platelets":263358.03,
               "serum_creatinine":1.1,
               "serum_sodium":136,
               "sex":1,
               "smoking":0,
               "time":6
               },
            {
               "age":60,
               "anaemia":0,
               "creatinine_phosphokinase":68,
               "diabetes":0,
               "ejection_fraction":20,
               "high_blood_pressure":0,
               "platelets":119000,
               "serum_creatinine":2.9,
               "serum_sodium":127,
               "sex":1,
               "smoking":1,
               "time":64
                }
        ]
        }
# data = {"data":
#         [
#             [55, 0, 7861, 0, 38, 0, 263358.03, 1.1, 136, 1, 0, 6],
#             [60, 0, 68, 0, 20, 0, 119000, 2.9, 127, 1, 1, 64]
               
#         ]
#         }

#d=pd.DataFrame(data)
#d=d.to_dict()
# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {primary}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

"{\"result\": [1, 1]}"


TODO: In the cell below, print the logs of the web service and delete the service

In [48]:
#logs of the webservice
logs=service.get_logs()
print(logs)

2021-02-01T22:13:52,290050747+00:00 - iot-server/run 
2021-02-01T22:13:52,316068390+00:00 - rsyslog/run 
2021-02-01T22:13:52,316201992+00:00 - gunicorn/run 
2021-02-01T22:13:52,317407703+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [49]:
#delete the deployed webservice
service.delete()